<a href="https://colab.research.google.com/github/Janani-SB/GenAI_projects-/blob/main/Conversational_RAG_Chatbot_using_LangChain_%26_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conversational RAG Chatbot using LangChain & OpenAI**

### Load Dependencies

In [ ]:
!pip install langchain==0.1.16
!pip install langchain-openai==0.1.3
!pip install langchain-community==0.0.33

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successfully uninstalled langchain-community-0.0.38


In [ ]:
!pip install langchain-chroma==0.1.0
!pip install langchainhub==0.1.15

  Using cached langchainhub-0.1.15-py3-none-any.whl.metadata (621 bytes)
  Using cached types_requests-2.32.4.20250611-py3-none-any.whl.metadata (2.1 kB)
Using cached langchainhub-0.1.15-py3-none-any.whl (4.6 kB)
Using cached types_requests-2.32.4.20250611-py3-none-any.whl (20 kB)


## Enter API Tokens

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass()

··········


if using Azure Open AI you might need to configure it based on how it is setup in your org.

Refer to [this](https://python.langchain.com/docs/integrations/llms/azure_openai/) for more details

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Load Wikipedia Data

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
import gzip
import json
import requests
from tqdm import tqdm
import sys


# Download a file from a URL
def http_get(url, path) -> None:
    """
    Downloads a URL to a given path on disc
    """
    if os.path.dirname(path) != "":
        os.makedirs(os.path.dirname(path), exist_ok=True)

    req = requests.get(url, stream=True)
    if req.status_code != 200:
        print("Exception when trying to download {}. Response {}".format(url, req.status_code), file=sys.stderr)
        req.raise_for_status()
        return

    download_filepath = path + "_part"
    with open(download_filepath, "wb") as file_binary:
        content_length = req.headers.get("Content-Length")
        total = int(content_length) if content_length is not None else None
        progress = tqdm(unit="B", total=total, unit_scale=True)
        for chunk in req.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                progress.update(len(chunk))
                file_binary.write(chunk)

    os.rename(download_filepath, path)
    progress.close()


In [ ]:
wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

100%|██████████| 50.2M/50.2M [00:11<00:00, 4.50MB/s]


In [ ]:
import gzip
import json

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        #Add all paragraphs
        #passages.extend(data['paragraphs'])

        #Only add the first paragraph
        passages.append(data['paragraphs'][0])

print("Passages:", len(passages))

Passages: 169597


In [ ]:
passages = [passage for passage in passages for x in ['india','cheetah','flying fish'] if x in passage.lower().split()]
len(passages)

778

# Load OpenAI LLMs

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

In [ ]:
from langchain_openai import OpenAIEmbeddings
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

# Generate LLM Embeddings and store them in Chroma Vector DB




In [ ]:
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain.docstore.document import Document
docs = [Document(page_content=doc) for doc in passages]

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
chunked_docs = splitter.split_documents(docs)

# Create Vector DB and Retriever

In [ ]:
chroma_db = Chroma.from_documents(documents=chunked_docs, collection_name='wiki_db', embedding = openai_embed_model,
                               collection_metadata = {"hnsw:space":"cosine"}, persist_directory="./wiki_db")

In [ ]:
chroma_db

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type='similarity_score_threshold', search_kwargs={"k":5, "score_threshold":0.3})

## Building the QA RAG Chain with Chat History

### Components of the QA RAG Chain

**Creating Document Chains:**
   

**Building the Final QA RAG Chain:**
  


# Conversational RAG System with LangChain






In [ ]:
from langchain import hub

In [ ]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = """You are an assistant for question-answering tasks.
use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
keep the answer upto 5 lines unless the user asks for more information

Question:
{question}

Context:
{context}

Answer:
"""

prompt_template = ChatPromptTemplate.from_template(prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_rag_chain = ({"context": (similarity_retriever | format_docs), "question":RunnablePassthrough()} | prompt_template | chatgpt)


In [ ]:
rephrase_prompt = hub.pull("langchain-ai/chat-langchain-rephrase")
rephrase_prompt

PromptTemplate(input_variables=['chat_history', 'input'], metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'chat-langchain-rephrase', 'lc_hub_commit_hash': 'fb7ddb56be11b2ab10d176174dae36faa2a9a6ba13187c8b2b98315f6ca7d136'}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.\n\nChat History:\n{chat_history}\nFollow Up Input: {input}\nStandalone Question:')

In [ ]:
print(rephrase_prompt.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {input}
Standalone Question:


## Contextualizing the Question



### Defining a Sub-Chain for Historical Context


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
rephrase_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history,
formulate a standalone question which can be understood without the chat history. DO NOT answer the question, just reformulate it if
needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages([("system",rephrase_system_prompt),
                                                    MessagesPlaceholder("chat_history"),
                                                    ("human", "{input}"),])
history_aware_retriever = create_history_aware_retriever(chatgpt, similarity_retriever, rephrase_prompt)

history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7b6662c09290>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history,\nformulate a standalone question which can be

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
keep the answer upto 5 lines unless the user asks for more information

Context:
{context}
"""

qa_prompt = ChatPromptTemplate.from_messages([("system",qa_system_prompt),
                                                    MessagesPlaceholder("chat_history"),
                                                    ("human", "{input}"),])

question_answer_chain = create_stuff_documents_chain(chatgpt, qa_prompt)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
qa_rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7b6662c09290>, search_type='similarity_score_threshold', search_kwargs={'k': 5, 'score_threshold': 0.3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference contex

In [ ]:
chat_history = []

question = "What is the capital of India?"
response = qa_rag_chain.invoke({"input":question, "chat_history":chat_history})
print(response['answer'])

The capital of India is New Delhi.


In [ ]:
for chunk in qa_rag_chain.stream({"input":question, "chat_history":chat_history}):
    print(chunk)

{'input': 'What is the capital of India?', 'chat_history': []}
{'context': [Document(page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'), Document(page_content='The Republic of India is divided into twenty-eight States,and eight union territories including the National Capital Territory.'), Document(page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as de

In [ ]:
chat_history

[]

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from langchain_core.messages import HumanMessage, AIMessage
chat_history.extend([HumanMessage(content=question), AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is the capital of India?'),
 AIMessage(content='The capital of India is New Delhi.')]

In [ ]:
question = "Tell me more about the city?"
response = qa_rag_chain.invoke({"input":question, "chat_history":chat_history})
print(response['answer'])

New Delhi is the capital of India and a union territory within the megacity of Delhi. It has a rich history and is home to several monuments. The city covers an area of about 42.7 km and has a population of approximately 9.4 million people. New Delhi is known for its expensive living costs and falls under the North Indian geographical zone.


In [ ]:
chat_history.extend([HumanMessage(content=question), AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is the capital of India?'),
 AIMessage(content='The capital of India is New Delhi.'),
 HumanMessage(content='Tell me more about the city?'),
 AIMessage(content='New Delhi is the capital of India and a union territory within the megacity of Delhi. It has a rich history and is home to several monuments. The city covers an area of about 42.7 km and has a population of approximately 9.4 million people. New Delhi is known for its expensive living costs and falls under the North Indian geographical zone.')]

In [ ]:
question = "Can fish really fly?"
response = qa_rag_chain.invoke({"input":question, "chat_history":chat_history})
print(response['answer'])

Fish cannot truly fly like birds, but some species, like the flying fish, can glide above the water's surface for short distances. They achieve this by spreading their fins and using their tails to propel themselves out of the water, allowing them to glide through the air to escape predators. However, this is not the same as flying in the traditional sense.


In [ ]:
response

{'input': 'Can fish really fly?',
 'chat_history': [HumanMessage(content='What is the capital of India?'),
  AIMessage(content='The capital of India is New Delhi.'),
  HumanMessage(content='Tell me more about the city?'),
  AIMessage(content='New Delhi is the capital of India and a union territory within the megacity of Delhi. It has a rich history and is home to several monuments. The city covers an area of about 42.7 km and has a population of approximately 9.4 million people. New Delhi is known for its expensive living costs and falls under the North Indian geographical zone.')],
 'context': [Document(page_content='The flying snake, or "Chrysopelea", is a mildly venomous snake found throughout India to the Indonesian archipelago. It can glide, in an arboreal habitat, going from tree to tree, most likely, much like the draco lizard. They\'re better at "flying" than another species of animal similar to this - the flying squirrel.'),
  Document(page_content='Archerfish (or archer fish)

In [ ]:
chat_history.extend([HumanMessage(content=question), AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is the capital of India?'),
 AIMessage(content='The capital of India is New Delhi.'),
 HumanMessage(content='Tell me more about the city?'),
 AIMessage(content='New Delhi is the capital of India and a union territory within the megacity of Delhi. It has a rich history and is home to several monuments. The city covers an area of about 42.7 km and has a population of approximately 9.4 million people. New Delhi is known for its expensive living costs and falls under the North Indian geographical zone.'),
 HumanMessage(content='Can fish really fly?'),
 AIMessage(content="Fish cannot truly fly like birds, but some species, like the flying fish, can glide above the water's surface for short distances. They achieve this by spreading their fins and using their tails to propel themselves out of the water, allowing them to glide through the air to escape predators. However, this is not the same as flying in the traditional sense.")]

In [ ]:
question = "What is the fastest animal?"
response = qa_rag_chain.invoke({"input":question, "chat_history":chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=response["answer"])])
chat_history

[HumanMessage(content='What is the capital of India?'),
 AIMessage(content='The capital of India is New Delhi.'),
 HumanMessage(content='Tell me more about the city?'),
 AIMessage(content='New Delhi is the capital of India and a union territory within the megacity of Delhi. It has a rich history and is home to several monuments. The city covers an area of about 42.7 km and has a population of approximately 9.4 million people. New Delhi is known for its expensive living costs and falls under the North Indian geographical zone.'),
 HumanMessage(content='Can fish really fly?'),
 AIMessage(content="Fish cannot truly fly like birds, but some species, like the flying fish, can glide above the water's surface for short distances. They achieve this by spreading their fins and using their tails to propel themselves out of the water, allowing them to glide through the air to escape predators. However, this is not the same as flying in the traditional sense."),
 HumanMessage(content='What is the 

In [ ]:
question = "Tell me about its different species?"
response = qa_rag_chain.invoke({"input":question, "chat_history":chat_history})
chat_history.extend([HumanMessage(content=question), AIMessage(content=response["answer"])])
print(response['answer'])

There are several subspecies of cheetah:

1. **South African Cheetah (Acinonyx jubatus jubatus)**: The most abundant subspecies, native to Southern Africa, with over 6,000 individuals in the wild.

2. **Asiatic Cheetah (Acinonyx jubatus venaticus)**: A critically endangered subspecies found in Asia, with a very small population remaining.

These subspecies differ in their geographic distribution and population status.
